# 🧩 Optimización SQL y Particionado de Datos

Objetivo: entender y aplicar técnicas de optimización de consultas (índices, estadísticas, JOIN strategies) y estrategias de particionado (bases relacionales y data lakes) para mejorar desempeño y costos.

- Duración: 90–120 min
- Dificultad: Media
- Prerrequisitos: SQL intermedio, nociones de modelado y almacenamiento columnar

### 🚀 **Query Optimization: Del Problema al Performance**

**¿Por qué Optimizar?**
- Query lento = Costos altos (Athena cobra por TB escaneado)
- Timeouts en dashboards → Mala UX
- Lock contention → Bloqueos en OLTP
- Escalabilidad limitada

**Jerarquía de Optimización:**

```
1. Schema Design (40% impacto)
   ├─ Normalización vs Desnormalización
   ├─ Tipos de datos correctos (INT vs VARCHAR)
   └─ Particionamiento estratégico

2. Índices (30% impacto)
   ├─ B-Tree para lookups
   ├─ Columnar para analytics
   └─ Covering indexes

3. Query Rewrite (20% impacto)
   ├─ Predicados pushdown
   ├─ JOIN order optimization
   └─ Eliminar subqueries correlacionadas

4. Hardware/Config (10% impacto)
   ├─ Memoria (work_mem, buffer pool)
   ├─ Parallel workers
   └─ Connection pooling
```

**Metodología de Optimización:**

1. **Measure**: EXPLAIN ANALYZE (costos reales)
2. **Identify**: Seq Scan largo, Hash Join costoso
3. **Hypothesize**: "Falta índice en columna X"
4. **Test**: CREATE INDEX y re-medir
5. **Validate**: A/B test en producción

**Costos en Query Plans:**
```sql
EXPLAIN (ANALYZE, BUFFERS) SELECT ...

Seq Scan on ventas  (cost=0.00..180.00 rows=10000 width=42) (actual time=0.05..5.32 rows=9856)
                     ^^^^^^^^^^^^^^^^    ^^^^^^^^^^^^^        ^^^^^^^^^^^^^^^^^^^^
                     Estimado            Estimado             REAL (con ANALYZE)
```

**Métricas Clave:**
- **Rows**: Estimado vs real (si difieren mucho → ANALYZE tabla)
- **Cost**: Unidades arbitrarias (comparar entre planes)
- **Time**: ms reales (actual time)
- **Buffers**: Bloques leídos (shared hit=cache, read=disk)

**Red Flags:**
- 🚩 Seq Scan en tablas >1M filas
- 🚩 Nested Loop con outer side > 10K filas
- 🚩 Sort en memoria → Disk (work_mem insuficiente)
- 🚩 Rows estimado != rows actual (10x+ diferencia)

---
**Autor:** Luis J. Raigoso V. (LJRV)

## 1. EXPLAIN / EXPLAIN ANALYZE (demo con SQLite)

### 🔍 **EXPLAIN ANALYZE: Decodificando Query Plans**

**Tipos de Scans:**

1. **Sequential Scan (Seq Scan):**
   ```
   Seq Scan on ventas  (cost=0.00..180.00 rows=10000)
   ```
   - Lee tabla completa, fila por fila
   - **Cuándo es OK**: Tablas pequeñas (<100K filas), queries que leen >25% de la tabla
   - **Malo**: Tablas grandes con filtros selectivos

2. **Index Scan:**
   ```
   Index Scan using idx_cliente on ventas  (cost=0.42..8.44 rows=1)
   ```
   - Lee índice → busca filas en tabla
   - **Cuándo es OK**: Filtros selectivos (<5% filas)
   - **Malo**: Range queries grandes (muchos random I/O)

3. **Index Only Scan:**
   ```
   Index Only Scan using idx_covering on ventas  (cost=0.42..4.44 rows=1)
   ```
   - Lee **solo** índice (covering index con INCLUDE)
   - **Mejor performance**: Sin acceso a tabla
   - Requiere: VACUUM para visibility map

4. **Bitmap Index Scan:**
   ```
   Bitmap Heap Scan on ventas
     Recheck Cond: (cliente_id = 10)
     -> Bitmap Index Scan on idx_cliente
   ```
   - Crea bitmap de bloques a leer → lee secuencialmente
   - **Cuándo**: Multiple index merge, rangos medianos

**Tipos de JOINs:**

1. **Nested Loop (NL):**
   ```
   Nested Loop  (cost=0.00..100.00 rows=10)
     -> Seq Scan on pequeña  (rows=10)
     -> Index Scan on grande  (rows=1)
   ```
   - Para cada fila de outer → busca en inner
   - **Óptimo**: Outer pequeño (<100 filas) + índice en inner
   - **Malo**: Outer grande sin índice en inner

2. **Hash Join:**
   ```
   Hash Join  (cost=120.00..500.00 rows=1000)
     Hash Cond: (t1.id = t2.id)
     -> Seq Scan on t1
     -> Hash
       -> Seq Scan on t2
   ```
   - Construye hash table de tabla pequeña en memoria
   - **Óptimo**: Joins equi (=) con tablas medianas
   - **Malo**: Hash table no cabe en work_mem → spill to disk

3. **Merge Join:**
   ```
   Merge Join  (cost=200.00..400.00 rows=1000)
     Merge Cond: (t1.id = t2.id)
     -> Sort ...
     -> Sort ...
   ```
   - Requiere tablas ordenadas
   - **Óptimo**: Ambas tablas con índice en join key
   - **Uso**: Joins en data warehouse con datos pre-ordenados

**Interpreting Costs:**
```sql
Nested Loop  (cost=0.42..123.45 rows=100)
              ^^^^^^^^^^^^^^^^
              0.42 = startup cost (setup)
              123.45 = total cost (incluye startup)
```

**PostgreSQL Specific:**
```sql
EXPLAIN (ANALYZE, BUFFERS, VERBOSE) SELECT ...

Buffers: shared hit=8 read=120 dirtied=0 written=0
         ^^^^^^^^^^    ^^^^^^^^
         Cache hits    Disk reads (malo si alto)
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
import sqlite3, pandas as pd
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.executescript('''
CREATE TABLE ventas (
 id INTEGER PRIMARY KEY, fecha TEXT, cliente_id INT, producto_id INT, cantidad INT, precio REAL, total REAL
);
CREATE INDEX idx_ventas_cliente_fecha ON ventas (cliente_id, fecha);
''')
import random, datetime as dt
rows = [ (i, str(dt.date(2025,10,(i%28)+1)), random.randint(1,200), random.randint(100,110), random.randint(1,5), round(random.uniform(5,200),2)) for i in range(1,5001) ]
rows = [ (*r, r[4]*r[5]) for r in rows ]
cur.executemany('INSERT INTO ventas (id, fecha, cliente_id, producto_id, cantidad, precio, total) VALUES (?,?,?,?,?,?,?)', rows)
conn.commit()
list(cur.execute('EXPLAIN QUERY PLAN SELECT * FROM ventas WHERE cliente_id=10 AND fecha>="2025-10-10"').fetchall())[:5]

### 1.1 Impacto de índices compuestos y selectividad

### 📊 **Índices Compuestos: Selectividad y Orden**

**Selectividad:**  
Proporción de filas únicas en una columna.

```
Alta selectividad:   email (99% único)      → Excelente para índice
Media selectividad:  ciudad (50 valores)    → Útil combinado
Baja selectividad:   género (M/F/O)         → Inútil solo
```

**Orden en Índices Compuestos:**

```sql
CREATE INDEX idx_bad ON ventas (genero, email);
CREATE INDEX idx_good ON ventas (email, genero);

-- Query: WHERE email = 'user@example.com' AND genero = 'F'

idx_bad:  Escanea todos M/F/O, luego filtra email ❌
idx_good: Busca email directo (selectivo), luego filtra genero ✅
```

**Regla: Columnas MÁS selectivas primero**

**Left-to-Right Usage:**
```sql
CREATE INDEX idx ON ventas (cliente_id, fecha, producto_id);

✅ WHERE cliente_id = 10
✅ WHERE cliente_id = 10 AND fecha > '2025-01-01'
✅ WHERE cliente_id = 10 AND fecha > '2025-01-01' AND producto_id = 5
❌ WHERE fecha > '2025-01-01'  -- No usa índice (falta cliente_id)
❌ WHERE producto_id = 5        -- No usa índice (faltan primeras columnas)
```

**Funciones Rompen Índices:**
```sql
-- ❌ No usa índice
SELECT * FROM users WHERE LOWER(email) = 'user@example.com';

-- ✅ Índice funcional (PostgreSQL)
CREATE INDEX idx_email_lower ON users (LOWER(email));

-- ✅ O mejor: normalizar dato al insertar
INSERT INTO users (email) VALUES (LOWER('User@Example.com'));
```

**LIKE Patterns:**
```sql
-- ✅ Usa índice B-Tree
WHERE email LIKE 'user@%'

-- ❌ No usa índice B-Tree (leading wildcard)
WHERE email LIKE '%@example.com'

-- ✅ Para leading wildcard: trigram index (PostgreSQL)
CREATE INDEX idx_email_trgm ON users USING gin (email gin_trgm_ops);
```

**Índice Parcial (Filtered Index):**
```sql
-- Solo indexar registros activos (ahorra espacio)
CREATE INDEX idx_active_users ON users (email) WHERE status = 'active';

-- Query DEBE incluir predicado
SELECT * FROM users WHERE email = 'x' AND status = 'active';  -- Usa índice
SELECT * FROM users WHERE email = 'x';                         -- No usa
```

**Maintenance:**
```sql
-- PostgreSQL: Bloat en índices tras muchos UPDATE/DELETE
REINDEX INDEX idx_ventas_cliente_fecha;

-- Análisis de bloat
SELECT schemaname, tablename, 
       pg_size_pretty(pg_total_relation_size(schemaname||'.'||tablename)) as size
FROM pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY pg_total_relation_size(schemaname||'.'||tablename) DESC;
```

**Trade-offs:**
- ✅ Índices: Aceleran SELECT/WHERE/JOIN
- ❌ Índices: Ralentizan INSERT/UPDATE/DELETE (mantenimiento)
- ❌ Índices: Ocupan espacio en disco
- **Regla**: Max 5-7 índices por tabla OLTP, más en OLAP

---
**Autor:** Luis J. Raigoso V. (LJRV)

- Use índices compuestos en el orden de los predicados más selectivos.
- Evite funciones sobre columnas indexadas en filtros (rompe el índice).
- Mantenga estadísticas actualizadas (ANALYZE).

## 2. Particionado en PostgreSQL (DDL de referencia)

### 🗂️ **Particionado en PostgreSQL: Estrategias**

**¿Por qué Particionar?**

1. **Query Performance**: Partition pruning reduce datos escaneados
2. **Maintenance**: DROP partición antigua vs DELETE millones de filas
3. **Parallelism**: Queries sobre particiones diferentes en paralelo
4. **Archival**: Mover particiones antiguas a tablespaces baratos

**Tipos de Particionado:**

1. **RANGE (Más común):**
   ```sql
   CREATE TABLE events (
     id BIGSERIAL,
     timestamp TIMESTAMPTZ NOT NULL,
     data JSONB
   ) PARTITION BY RANGE (timestamp);
   
   -- Particiones mensuales
   CREATE TABLE events_2025_10 PARTITION OF events 
     FOR VALUES FROM ('2025-10-01') TO ('2025-11-01');
   
   CREATE TABLE events_2025_11 PARTITION OF events 
     FOR VALUES FROM ('2025-11-01') TO ('2025-12-01');
   ```
   
   **Uso**: Fechas, IDs secuenciales

2. **LIST:**
   ```sql
   CREATE TABLE orders (
     id BIGSERIAL,
     country VARCHAR(2),
     amount NUMERIC
   ) PARTITION BY LIST (country);
   
   CREATE TABLE orders_us PARTITION OF orders FOR VALUES IN ('US');
   CREATE TABLE orders_eu PARTITION OF orders FOR VALUES IN ('DE', 'FR', 'ES');
   ```
   
   **Uso**: Categorías discretas (país, región, status)

3. **HASH:**
   ```sql
   CREATE TABLE users (
     id BIGSERIAL,
     email VARCHAR
   ) PARTITION BY HASH (id);
   
   CREATE TABLE users_0 PARTITION OF users FOR VALUES WITH (MODULUS 4, REMAINDER 0);
   CREATE TABLE users_1 PARTITION OF users FOR VALUES WITH (MODULUS 4, REMAINDER 1);
   ```
   
   **Uso**: Distribución uniforme cuando no hay columna obvia

**Partition Pruning:**
```sql
-- Query escanea SOLO partición 2025_10
EXPLAIN SELECT * FROM events WHERE timestamp >= '2025-10-15';

Result:
  -> Seq Scan on events_2025_10  -- ✅ Solo 1 partición
     Filter: (timestamp >= '2025-10-15')
```

**Sin pruning:**
```sql
-- ❌ Escanea TODAS las particiones (función rompe pruning)
SELECT * FROM events WHERE EXTRACT(YEAR FROM timestamp) = 2025;

-- ✅ Reescribir para habilitar pruning
SELECT * FROM events WHERE timestamp >= '2025-01-01' AND timestamp < '2026-01-01';
```

**Índices en Particiones:**
```sql
-- Opción 1: Índice por partición (manual)
CREATE INDEX ON events_2025_10 (timestamp);
CREATE INDEX ON events_2025_11 (timestamp);

-- Opción 2: Índice global (PostgreSQL 11+, automático)
CREATE INDEX ON events (timestamp);
-- Crea índices en todas las particiones actuales y futuras
```

**Maintenance:**
```sql
-- Agregar partición nueva (automatizar con cron)
CREATE TABLE events_2025_12 PARTITION OF events 
  FOR VALUES FROM ('2025-12-01') TO ('2026-01-01');

-- Archivar partición antigua (instántaneo)
ALTER TABLE events DETACH PARTITION events_2024_01;
-- Ahora events_2024_01 es tabla independiente
ALTER TABLE events_2024_01 RENAME TO events_2024_01_archive;

-- Eliminar partición (cuidado!)
DROP TABLE events_2024_01;  -- ⚠️ Datos perdidos
```

**Limitaciones:**
- Primary key DEBE incluir partition key
- Foreign keys complicados (solo dentro de partición)
- UNIQUE constraints deben incluir partition key

**Partición Automática (pg_partman):**
```sql
-- Extensión para crear/eliminar particiones automáticamente
CREATE EXTENSION pg_partman;

SELECT create_parent('public.events', 'timestamp', 'native', 'monthly');
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
ddl = '''
-- Tabla particionada por rango de fecha
CREATE TABLE ventas_part (
  id BIGSERIAL PRIMARY KEY,
  fecha DATE NOT NULL,
  cliente_id INT,
  producto_id INT,
  cantidad INT,
  total NUMERIC(12,2)
) PARTITION BY RANGE (fecha);
-- Particiones mensuales
CREATE TABLE ventas_2025_10 PARTITION OF ventas_part FOR VALUES FROM ('2025-10-01') TO ('2025-11-01');
CREATE INDEX ON ventas_2025_10 (fecha, cliente_id);
ANALYZE ventas_2025_10;
-- Consulta con pruning de particiones
EXPLAIN ANALYZE SELECT SUM(total) FROM ventas_part WHERE fecha >= '2025-10-10' AND fecha < '2025-10-20';
'''
print(ddl)

## 3. Data Lakes: particionado y pruning (Parquet)

### 🏔️ **Data Lake Partitioning: Hive-style y Pruning**

**Hive-style Partitioning:**
```
s3://datalake/events/
├── year=2025/
│   ├── month=10/
│   │   ├── day=01/
│   │   │   ├── part-00000.parquet
│   │   │   └── part-00001.parquet
│   │   └── day=02/
│   └── month=11/
└── year=2024/
```

**Nomenclatura Estándar:**
```
{table}/key1=value1/key2=value2/.../file.parquet

✅ events/year=2025/month=10/day=01/data.parquet
❌ events/2025/10/01/data.parquet  -- No es Hive-style (sin key=value)
```

**Beneficios:**

1. **Partition Pruning Automático:**
   ```sql
   -- Athena/Spark/Trino detectan particiones sin metadata
   SELECT * FROM events WHERE year = 2025 AND month = 10;
   -- Escanea solo s3://datalake/events/year=2025/month=10/
   ```

2. **Ahorro de Costos:**
   ```
   Sin particionado: 1 TB escaneado = $5 (Athena)
   Con particionado:  10 GB escaneado = $0.05 (99% ahorro)
   ```

3. **Query Performance:**
   - Menos datos leídos = Menos tiempo
   - Paralelización por partición

**Columnas de Particionado (Orden importa):**

```python
# ✅ Bueno: Alta a baja cardinalidad
df.write.partitionBy("year", "month", "day", "hour").parquet("s3://...")
# Genera: year=2025/month=10/day=30/hour=14/

# ❌ Malo: Baja cardinalidad primero
df.write.partitionBy("country", "year", "month").parquet("...")
# country solo tiene ~200 valores → menos paralelización

# ❌ Muy Malo: Alta cardinalidad extrema
df.write.partitionBy("user_id").parquet("...")
# Millones de particiones → metadata overhead
```

**Small Files Problem:**
```
❌ Malo: 10,000 archivos de 1 MB = 10 GB
✅ Bueno: 80 archivos de 128 MB = 10 GB

Por qué:
- Overhead de listar S3 (1 API call por archivo)
- Spark tasks ineficientes (1 task por archivo)
- Metadata explosion en Glue Catalog
```

**Solución: Compaction**
```python
# Spark repartition antes de escribir
df.repartition(80).write.parquet("s3://...")

# O COALESCE (reduce particiones sin shuffle)
df.coalesce(80).write.parquet("s3://...")

# Target: 128-512 MB por archivo
```

**Formato Columnar (Parquet):**
```
Row-oriented (CSV):
[id, name, age, country]
[1, "Alice", 30, "US"]
[2, "Bob", 25, "UK"]
↓ Query: SELECT age FROM users
↓ Lee TODO (3 columnas innecesarias)

Column-oriented (Parquet):
[id]:      [1, 2]
[name]:    ["Alice", "Bob"]
[age]:     [30, 25]  ← Lee SOLO esta columna
[country]: ["US", "UK"]
```

**Ventajas Parquet:**
- ✅ Compresión columnar (10x+ vs CSV)
- ✅ Predicate pushdown (skip row groups)
- ✅ Schema evolution (agregar columnas)
- ✅ Tipos de datos nativos (no strings)

**Compresión:**
```python
df.write.option("compression", "snappy").parquet("...")

Snappy:  Rápido, baja compresión (~2x)
GZIP:    Lento, alta compresión (~4x)
ZSTD:    Balance (3x, speed OK)
```

**Z-Ordering (Databricks/Delta Lake):**
```sql
OPTIMIZE events ZORDER BY (user_id, event_type)
-- Co-localiza datos frecuentemente filtrados juntos
-- Mejora data skipping (skip más archivos)
```

**Partitioning Anti-patterns:**
```
❌ Particionar por columna con alta cardinalidad (user_id)
❌ Más de 4 niveles de particionado (overhead)
❌ Particiones con <10 MB (too small)
❌ Particiones con >10 GB (too large, no paralelismo)
```

---
**Autor:** Luis J. Raigoso V. (LJRV)

- Particionar por columnas de alta cardinalidad temporal (por ejemplo, año/mes/día).
- Ajustar tamaño de archivos (128–512 MB) para evitar small files problem.
- Pruning: motores como Spark/Athena/Trino escanean solo particiones necesarias.
- Usa formato columnar (Parquet) y compresión (Snappy/ZSTD).

## 4. Window functions y agregaciones eficientes

### 📈 **Window Functions: Optimización de Análisis Temporal**

**Window Functions vs GROUP BY:**

```sql
-- GROUP BY: Agrega y colapsa filas
SELECT cliente_id, SUM(total) 
FROM ventas 
GROUP BY cliente_id;
-- Resultado: 1 fila por cliente

-- Window Function: Mantiene todas las filas + columna calculada
SELECT cliente_id, fecha, total,
       SUM(total) OVER (PARTITION BY cliente_id) as total_cliente
FROM ventas;
-- Resultado: Todas las filas originales + total_cliente
```

**Sintaxis:**
```sql
<función_ventana> OVER (
  PARTITION BY <columnas>    -- Opcional: grupos
  ORDER BY <columnas>        -- Opcional: orden dentro de grupo
  ROWS/RANGE BETWEEN ...     -- Opcional: frame de ventana
)
```

**Funciones Comunes:**

1. **Ranking:**
   ```sql
   SELECT producto_id, ventas,
          ROW_NUMBER() OVER (ORDER BY ventas DESC) as rn,
          RANK() OVER (ORDER BY ventas DESC) as rank,
          DENSE_RANK() OVER (ORDER BY ventas DESC) as dense_rank
   FROM productos;
   
   -- ROW_NUMBER: 1,2,3,4,5 (sin empates)
   -- RANK:       1,2,2,4,5 (con gaps)
   -- DENSE_RANK: 1,2,2,3,4 (sin gaps)
   ```

2. **Agregaciones Acumulativas:**
   ```sql
   SELECT fecha, ventas,
          SUM(ventas) OVER (ORDER BY fecha 
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as acumulado
   FROM ventas_diarias;
   ```

3. **Moving Averages:**
   ```sql
   -- Promedio móvil últimos 7 días
   SELECT fecha, temperatura,
          AVG(temperatura) OVER (ORDER BY fecha 
                                 ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as avg_7d
   FROM clima;
   ```

4. **LAG/LEAD (Comparar con fila anterior/siguiente):**
   ```sql
   SELECT fecha, precio,
          LAG(precio, 1) OVER (ORDER BY fecha) as precio_anterior,
          precio - LAG(precio, 1) OVER (ORDER BY fecha) as cambio
   FROM stock_prices;
   ```

**Frame Specifications:**

```sql
ROWS BETWEEN:  -- Físico (conteo de filas)
  UNBOUNDED PRECEDING  -- Desde inicio
  3 PRECEDING          -- 3 filas atrás
  CURRENT ROW          -- Fila actual
  2 FOLLOWING          -- 2 filas adelante
  UNBOUNDED FOLLOWING  -- Hasta final

RANGE BETWEEN: -- Lógico (valores)
  -- Ejemplo: RANGE BETWEEN INTERVAL '1 day' PRECEDING AND CURRENT ROW
```

**Optimización:**

1. **Índice en ORDER BY:**
   ```sql
   CREATE INDEX idx_fecha ON ventas (fecha);
   
   -- Window function se beneficia del orden
   SELECT *, SUM(total) OVER (ORDER BY fecha) FROM ventas;
   ```

2. **Multiple Windows (share window):**
   ```sql
   -- ❌ Malo: Define ventana 3 veces
   SELECT 
     SUM(x) OVER (PARTITION BY y ORDER BY z),
     AVG(x) OVER (PARTITION BY y ORDER BY z),
     MAX(x) OVER (PARTITION BY y ORDER BY z)
   FROM t;
   
   -- ✅ Bueno: Define 1 vez, reutiliza
   SELECT 
     SUM(x) OVER w,
     AVG(x) OVER w,
     MAX(x) OVER w
   FROM t
   WINDOW w AS (PARTITION BY y ORDER BY z);
   ```

3. **Evitar Window en WHERE:**
   ```sql
   -- ❌ No funciona (window functions post-WHERE)
   SELECT * FROM ventas 
   WHERE SUM(total) OVER (PARTITION BY cliente_id) > 1000;
   
   -- ✅ Usar CTE o subquery
   WITH ranked AS (
     SELECT *, SUM(total) OVER (PARTITION BY cliente_id) as total_cliente
     FROM ventas
   )
   SELECT * FROM ranked WHERE total_cliente > 1000;
   ```

**Use Cases:**
- Top N por grupo (ROW_NUMBER + WHERE rn <= N)
- Running totals (cumulative sums)
- Year-over-year comparisons (LAG con PARTITION BY mes)
- Percentiles (PERCENTILE_CONT, NTILE)

**Performance:**
- Window functions requieren sort (costoso en tablas grandes)
- Considerar materializar resultados si se reutilizan frecuentemente
- Spark/Presto: Optimizan windows compartidos automáticamente

---
**Autor:** Luis J. Raigoso V. (LJRV)

In [ ]:
import pandas as pd
df = pd.read_sql_query('SELECT fecha, cliente_id, total FROM ventas', conn)
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.sort_values(['cliente_id','fecha'])
df['acumulado_cliente'] = df.groupby('cliente_id')['total'].cumsum()
df.head()

## 5. Buenas prácticas

- Evitar SELECT * en producción; proyectar solo columnas necesarias.
- Usar límites de tiempo y paginación en consultas intensivas.
- Mantener índices y particiones según patrones de acceso actuales.
- Monitorear planes y tiempos con muestras periódicas; automatizar regresiones.

---

## 🧭 Navegación

**← Anterior:** [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)

**Siguiente →:** [🚀 Servicios de Datos con FastAPI →](08_fastapi_servicios_datos.ipynb)

**📚 Índice de Nivel Mid:**
- [⚡ Mid - 01. Orquestación de Pipelines con Apache Airflow](01_apache_airflow_fundamentos.ipynb)
- [Streaming con Apache Kafka: Fundamentos](02_streaming_kafka.ipynb)
- [☁️ AWS para Ingeniería de Datos: S3, Glue, Athena y Lambda](03_cloud_aws.ipynb)
- [☁️ GCP para Ingeniería de Datos: BigQuery, Cloud Storage, Dataflow y Composer](03b_cloud_gcp.ipynb)
- [☁️ Azure para Ingeniería de Datos: ADLS, Synapse, Data Factory y Databricks](03c_cloud_azure.ipynb)
- [🗄️ Bases de Datos Relacionales y NoSQL: PostgreSQL y MongoDB](04_bases_datos_postgresql_mongodb.ipynb)
- [♻️ DataOps y CI/CD para Pipelines de Datos](05_dataops_cicd.ipynb)
- [🌐 Conectores Avanzados: REST, GraphQL y SFTP](06_conectores_avanzados_rest_graphql_sftp.ipynb)
- [🧩 Optimización SQL y Particionado de Datos](07_optimizacion_sql_particionado.ipynb) ← 🔵 Estás aquí
- [🚀 Servicios de Datos con FastAPI](08_fastapi_servicios_datos.ipynb)
- [🧪 Proyecto Integrador Mid 1: API → DB → Parquet con Orquestación](09_proyecto_integrador_1.ipynb)
- [🔄 Proyecto Integrador Mid 2: Kafka → Streaming → Data Lake y Monitoreo](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
